In [1]:
!python -V

Python 3.12.7


In [14]:
import pandas as pd
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import mlflow

In [3]:
df = pd.read_parquet("/Users/muhammadwisalabdullah/Downloads/yellow_tripdata_2023-01.parquet")

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
print(df.tpep_dropoff_datetime - df.tpep_pickup_datetime)
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

print(df.duration)

0         0 days 00:08:26
1         0 days 00:06:19
2         0 days 00:12:45
3         0 days 00:09:37
4         0 days 00:10:50
                ...      
3066761   0 days 00:13:59
3066762   0 days 00:19:27
3066763   0 days 00:24:31
3066764   0 days 00:13:00
3066765   0 days 00:14:24
Length: 3066766, dtype: timedelta64[us]
0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64


In [6]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("NYC_FarePredictor")

<Experiment: artifact_location='/Users/muhammadwisalabdullah/PycharmProjects/MLOPsZoomcampHWs/HW3/mlruns/1', creation_time=1748937303092, experiment_id='1', last_update_time=1748937303092, lifecycle_stage='active', name='NYC_FarePredictor', tags={}>

In [9]:
def read_dataframe1(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


In [79]:
train_df = read_dataframe1("/Users/muhammadwisalabdullah/Downloads/yellow_tripdata_2023-01.parquet")
valid_df = read_dataframe1("/Users/muhammadwisalabdullah/Downloads/yellow_tripdata_2023-02.parquet")
print(train_df.shape)
print(valid_df.shape)

(3009173, 20)
(2855951, 20)


In [80]:
train_df['PU_DO'] = train_df['PULocationID'] + '_' + train_df['DOLocationID']
valid_df['PU_DO'] = valid_df['PULocationID'] + '_' + valid_df['DOLocationID']

In [82]:
valid_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,PU_DO
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,...,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333,142_163
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,...,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333,132_26
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,...,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000,161_145
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,...,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333,148_236
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,...,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000,137_244


In [77]:
categorical = ['PU_DO']

In [83]:
train_dicts = train_df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(f"Dimensionality after OHE: {X_train.shape[-1]}")

Dimensionality after OHE: 21801


In [84]:
target = 'duration'
y_train = train_df[target].values

In [85]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [86]:
y_pred = lr.predict(X_train)
import numpy as np

mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(rmse)

5.026739072111905


In [87]:
val_dicts = valid_df[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
print(f"Dimensionality after OHE: {X_val.shape[-1]}")

Dimensionality after OHE: 21801


In [88]:
target = 'duration'
y_val = valid_df[target].values
y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(rmse)

5.198560213447683
